# DCM Tutorial 

The dynamic causal model, first presented by Friston, et al. 2002, provides a means of modeling the effective connectivity between neural populations that underlies task based or resting state functional neuroimaging data as measured through fMRI, EEG, and MEG. 

## Model Components

The model consists of three main parts:

-  **Dynamic Causal Model:** A model that combines a bilinear model of neuronal dynamics with a neuroimaging method - specific model (based on hemodynamic or electrophysiological signatures). Proposed brain regions and potential connections between regions are specified in one or more candidate models.
-  **Variational Laplace:** The process by which candidate model parameters are optimized and their uncertainty is determined. Approximates the log marginal likelihood for each model using variational free energy. Comparing the free energy of candidate models enalbes the model that best explains the data to be selected.
-  **Model Selection & Comparison:** The process by which model structure is selected. There are two common approaches:
    - **Bayesian Model Selection (BMS):** Multiple DCM models are optimized, where each describes a hypothesis about the connectivity between brain regions of interest. The model that best describes each participant's data is determined and the average connectivity parameters are determined using Bayesian Model Averaging, with each parameter a weighted average of the candidate models based on each's prior probability.
    - **Parametric Empirical Bayes (PEB):** Assumes that each participant varies in the strength of connection between regions. Models the soruces of variability in connection strengths across subjects using fixed and random effects. Specifies one DCM per subject and a Bayesian GLM to model parameters at the group level. Tests hypotheses by comparing the group level model to reduced group level models where certain combinations of connections are switched on and off to determine the best fit for the data. 

## (1) General DCM Model: Mathematical Framework 

This model produces a deterministic model of neurla dynamics in a network of n interacting brain regions/nodes. 

The model transforms a neuronal state vector $\vec{x}$ where each region is represented by a single hidden state using the bilinear differential equation:

$\frac{dx}{dt} = f(x, u, \theta) = Ax + \Sigma_{j=1}^m u_j B^{(j)} x + Cu$ 

where

$u$ represents the external or exogenous input.

$A = \frac{\partial f}{\partial x} |_{u=0}$ represents the fixed (average) coupling among nodes in the absence of exogenous output $u(t)$. This roughly represents latent coupling among nodes in the absence of experimental perturbations.

$B = \frac{\partial^2 f}{\partial x \partial u}$ represents the change in latent coupling induced by the jth input, which encodes context sensitive changes in A, or modulations of coupling by experimental perturbations. 

$C = \frac{\partial f}{\partial u} |_{x = 0}$ represents the influence of exogneous inputs that cause perturbations in hidden states. 

The model results from a bilinear Taylor approximation to any dynamical model of how changes in neuronal activity in node x_i are caused by changes in the others. Parameters $A, B, C \in \theta$ are coupling parameters we seek to find that determine the functional architecture and interactions between brain regions at a neuronal level. 

### DCM for fMRI: Hemodynamic Model 

This model describes the transformation of neural activity into a BOLD response. The common hemodynamic model uses the "balloon model", which uses the analogy of an inflated balloon to describe the behavior of the postcapillary venous component of the blood vessels "feeding" the neurons to encourage activity. The model postulates a nonlinear dependence of the BOLD signal on blood volume $V$ and deoxyhemoglobin content $q$. It explores the evolution of $V$ and $q$ by two differential equations operating under the assumptions that: 

- Small post capillary blood vessels releact to an influx of blood like an inflating balloon. So, changes in normalized blood volume $V$ correspond to differences in flow $f_i$ and outflow $f_o$ with some time constant $\tau$ where $\tau$ represents the mean transit time for blood to traverse the venous compartment such that $ \tau = \frac{V_0}{f_0} = \frac{\text{resting blood volume}}{\text{resting blood flow}}$. Outflow $f_o$ is modelled as a function of volume with parameter $\alpha$ the resistance of the venous balloon: $f_o = v^{1/\alpha}$.

- Oxygen extraction is highly coupled to blood flow, so it determines the deoxyhemoglobin content $q$. Generally a change in $q$ refers to a delivery of deoxyhemoglobin into the venous compartment minus that expelled at a given time point. So, the oxygen gradient across the capillary wall and oxygen extraction rate depend entirely on oxygen delivery, and as a result, blood flow.

These assumptions lead to state equations for $V$ and $q$:

$\tau \frac{dV}{dt} = f_i(t) - v(t)^{1/\alpha}$

$\tau \frac{dq}{dt} = f_i(t) ( \frac{1 - (1 - E_0)^{1/f_i}}{E_0}) - v(t)^{1/\alpha}(\frac{q(t)}{(v(t)}$

where $E_0$ is oxygen extraction fraction at rest. 

The hemodynamic model also has vascular state equations that address how bloodflow depends on neural activity, connecting to the neural state vector $x$. These vascular responses to neural activity correspond to a damped oscillator: changes in neural activity $x$ elicit exponentially decaying vasodilatory signal $s$ subject to feedback regulation by the flow $f$ which is normalized to resting flow $F_0$ that it induces. This results in two equations:

$ \frac{ds}{dt} = x - \kappa s - \gamma(f-1)$

$\frac{df}{dt} = s$ 

where $x$ is the neural state vector, $\kappa$ the rate constant of signal decay, and $\gamma$ the rate constant of feedback regulation. 

Several variants of the BOLD model itself, which unites parameters determined by the prior equations, but all are variations on a general model proposed by Stephen et al. 2007: 

$ \frac{\delta s}{s_0} = v_0 ( k_1(1 - q) + k_2(1 - \frac{q}{v}) + k_3(1 - v))$ 

where 

$k_1 = 4.3 \upsilon_0 E_0 T E$

$k_2 = \epsilon r_0 E_0 TE$

$k_3 = 1 - \epsilon$

and $\upsilon_0$ is the frequency offset for fully deoxygenated blood ($\upsilon = 40.3 s^{-1}$ at 1.5 teslas, $\epsilon$ is the ratio of intra and exogenous signal, TE is the time elapsed between when protons are excited and measured (how long signal creation takes), and $s_0$ is the BOLD signal at rest, described by:

$s_0 = (1 - V_0)S_E + V_0S_I$

where $V_0$ is the resting venous volume fraction, $S_E$ is the extravenous signal, and $S_I$ the intravascular signal. 

For any combination of parameters $\theta$ and inputs $u$, the measured BOLD response $y$ is modelled as a predicted BOLD signal $h(x, u, \theta)$ plus a linear combination of confounds $X\beta$ and Gaussian observation error:

$y = h(x, u, \theta) + X\beta + \epsilon$

### DCM for EEG: Electrophysiological Model

The EEG adaption was originally presented in David, Kiebel, et al. 2006. In this adaptation, the neuronal part of DCM is represented by a graph of activity sources where each source is represented with three neuronal subpopulations representing forward, backward, and lateral connections. These populations are connected within themselves and to other populations. The connections between sources are based on the Jansen Model (1995), and subdivided into supragranular layers, infragranular layers, and granular layer 4 based on monkey cortex studies. 

The three classes of connections used are classified as:

- bottom up or froward connections originating in agranular layers and terminating in layer #4
- top down or backward connections connecting agranular layers
- lateral connections starting in agranular layers and targeting all layers

All connections are excitatory and assumed to occur between pyramidal cells.

#### Jansen Model 

In this model, excitatory pyramidal cells receive input from inhibitory and excitatory populations of interneurons via interal connections confined to the cortical sheet. Excitatory interneuorns are assumed to be spiny stellate cells primarily found in layer 4, receiving forward connections. Excitatory pyramidal cells and inhbiitory interneurons occupy agranular layers and receive backward and lateral inputs. 

Similar to the fMRI structure, the model is composed of two main equations:

$ \frac{dx}{dt} = f(x, u, \theta)$ 

$ h = g(x, \theta)$

where $\theta$ are unknown quantities that parametrize the state and observer equations, $x$ represents cortical neural states, and $u$ represents exogenous inputs, experimentally produced differences in activity above baseline. 

The state equations $\frac{dx}{dt} = f(x, u, \theta) $ for the neuronal states of the multiple areas used are:

$\frac{dx_0}{dt} = x_5 - x_6$

$\frac{dx_1}{dt} = x_4$

$\frac{dx_2}{dt} = x_5$

$\frac{dx_3}{dt} = x_6$

$\frac{dx_4}{dt} = \frac{H_e}{\tau_e} ((C^F + C^L + \gamma_1I) S(x_0) + C^Uu) - \frac{2x_4}{\tau_e} - \frac{x_1}{\tau_e^2}$

$\frac{dx_5}{dt} = \frac{H_e}{\tau_e} ((C^B + C^L) S(x_0) + \gamma_2S(x_1)) \frac{2x_5}{\tau_e} - \frac{x_2}{\tau_e^2}$

$\frac{dx_6}{dt} = \frac{H_i}{\tau_i} \gamma_4 S(x_7) - \frac{2x_6}{\tau_i} - \frac{x_3}{\tau_i^2}$

$\frac{dx_7}{dt} = x_8$

$\frac{dx_8}{dt} = \frac{H_e}{\tau_e} ((C^B + C^L + \gamma_3I) S(x_0)) - \frac{2x_8}{\tau_e} - \frac{x_7}{\tau_e}$


where $x_j = [x_j^{(1)} ...]^T$, 

$x_0^{(i)}, ..., x_8^{(i)}$ is the mean transmembrane potentials and currents of the 3 subpopulations from the ith source,

$C_F$ is the forward extrinsic connections, 

$C_B$ the backwards intrinsic connetions, 

$C_L$ the lateral extrinsic connections,

and $\gamma_{1,2,3,4}$ the constants that determine the strength of intrinsic connections and reflect the number of synapses expressed by each subpopulation. 

The other variables come from two operators that determine the dynamics of the system. 

## (2) Variational Laplace / Optimization

To review, DCMs generally involve two main equations: 

$ \frac{dx}{dt} = J(t) x(t) + C u_{i}(t) $,

with $J(t)$ the time-varying connectivity matrix and $u_i(t)$ the vector of external driving inputs and 

$ y - g(x, \theta_n) + \epsilon_y$ ,

an observation model that translates $x$ into fMRI (or EEG) activity, $y$. 

Our goal in defining these equations and possible network connectivities is to first optimize connectivity parameters for each of the suggested models, then decide which possible optimized model structure is the best candidate for that of our experimental data. This section describes how to optimize connectivity parameters in each candidate model structure using a form of maximum likelihood optimization known as Variational Laplace. 

### Setting Up for Variational Laplace 

Here we will briefly describe how to set up for the variational laplace technique. For a more indepth review, see Friston, et al. 2002 and xxx. 

We begin by defining:

$P(\theta)$: the prior, our belief of model parameters before "seeing" the data

$P( y | \theta)$: the likelihood, the likelihood of observing data $y$ given parameters $\theta$.

The likelihood and prior are related by Bayes' rule: 

$ P(y) = \int p(y, \theta) d \theta = \int p(y | \theta) p(\theta) d\theta $

This equation seems to provide a direct way for us to calculate $p(y)$, which is the probability of the data we have using an optimized model structure. Unfortunately this equation cannot be solved diretly, so we rely on variational Bayes methods to create an optimzed lower bound on the probability $p(y)$ so we know at least how well a given model can be performed. 

Using Bayesian inference to update the probability density over parameters after seeing the data allows us to obtain $p(y)$. The Variational Laplace technique involves utilizing "free energy" as an approximation to the marginal likelihood, which provides a means of scoring the models. 

To construct a lower bound on $p(\theta | y)$, we introduce a probability density $Q(\theta)$ which represents an approximation on the posterior over the parameters to yield:

$ \ln p(y) = \ln E_{Q(\theta)} \int \frac{P(y, \theta)}{Q(\theta)} d\theta$

We then define free energy $F(Q(\theta)) = E_{Q(\theta)} [ \ln \frac{p(y, \theta)}{Q(\theta)}]$, which using Jensen's inequality (the average of a log is $\leq$ the log of an averge) yields: 

$F[Q(\theta)] \leq \ln p(y) $

This implies that free energy provides a lower bound on the natural logarithm of $p(y)$. 

It can be demonstrated that 

$F[Q(\theta)] = E_{Q(\theta)} [ \ln P(y, \theta) - \ln Q(\theta)]$. 

To optimize parameters that comprise the free energy $\theta$, we develop expressions for $\ln p(y, \theta)$ and $Q(\theta)$ to yield: 

$ F[Q(\theta)] =  \ln p(y, \mu) - \frac{n}{2} + \frac{1}{2} [\ln(|\Sigma|) + n \ln 2\pi e]$ 

where $\mu = \text{argmax}_\theta p(y, \theta) = \text{argmax}_\theta p(\theta | y)$,

$n$ is the total number of parameters, and

$\Sigma$ the posterior covariance.

An alternative formulation of the free energy involves separating the models parameters $\beta$ and hyperparameters $\lambda$ that control observation noise. This alternative uses a mean field approximation to separate $\theta$  into $\beta$ and $\lambda$. 

### Maximizing Free Energy $ F[ Q(\theta)] $



The variational laplace technique involves continuously transitioning between Gradient Ascent and Gauss Newton optimization, enabling faster more accurate convergence to the optimal model parameters. The switching continues until the changein free energy between iterations is sufficiently small to be considered to have converged. 



## Model Selection and Comparison 

Several approaches to model selection and comparison exist. Two of the most common approaches for DCM fMRI model selection are Bayesian Model Selection (BMS) and Parametric Empirical Bayes (PEB). 

### Bayesian Model Selection (BMS)

This technique involves computation of the Bayes factor:

$ B_{ij} = \frac{p(Y | m = i)}{p(Y | m = j)} \approx \frac{F[\theta_i]}{F[\theta_i]}$

This compares the approximate posteriors of models i and j with a free energy approximation. 

The log Bayes factor is approximately the difference of model free energies: $ log( B_{ij}) \approx F_i - F_j$. 

It can be interpreted as evidence for a model or hypothesis. If there is not strong evidence for one model over the other by comparing Bayes factors, Bayesian Model Averaging can be used to determine the ideal model parameters by computing the weighted average of candidate model parameters based on each's prior probability. 

### Parametric Empirical Bayes (PEB)

The PEB approach enables the evaluation of group and within-subject variability on model parameters. 

Here, a single DCM is developed for each participant with all parameters of interest kept "on" and all parameters optimized using the methods outlined above. Then create a group level generalized linear model (GLM) over all parameters - this is the PEB model. 

This model can be used to test covariates and between subject effects. Hypotheses regarding model connectivity can be tested by comparing the "full" PEB with alternate PEBs with certain parameters switched off. The best model can be determined via comparison with variational laplace or if no model is a clear winner, Bayesian Model Averaging can be used. 

# Sample Code and Model Example

A MATLAB GUI implementation with a sample usage can be found [here](https://github.com/pzeidman/dcm-peb-example). This program is accompanied by two papers: [part 1](https://www.sciencedirect.com/science/article/pii/S1053811919305221) and [part 2](https://www.sciencedirect.com/science/article/pii/S1053811919305233) describing how DCM is used for fMRI. 

These papers describe a simple implementation of DCM for a fMRI neuroimaging study. DESCRIBE THE STUDY. A simple implementation, includng the development of a DCM model for a single participant's fMRI data from this study is shown below as an example implementation of DCM. 

Here, we use preprocessed data from subject 1 in the tutorial paper linked above to illustrate the functionality of the DCM model.

First, we determine the connectivity matrices for each participant, corresponding to which regions are turned off and turned on. 

In [3]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

In [5]:
# MRI settings
TR = 3.6    # Repetition time (secs)
TE = 0.05   # Echo Time (secs)

# Experiment settings
nsubjects = 60
nregions = 4
nconditions = 3

# Index of conditions in the DCM, for code readability
TASK = 1
PICTURES = 2
WORDS = 3

# Index of DCM regions, for code readability
lvF = 1
ldF = 2
rvF = 3
rdF = 4

In [7]:
# Determine connectivity via matrices A - C. 
# Columns are outgoing connections and rows incoming connections.
# Priors can be changed in these matrices...

# (A) represents the average connectivity across experimental conditions and matrix (a) is
# an indicator matrix for which of these connections we want switched on. 
A = np.array([[1, 1, 1, 0],
              [1, 1, 0, 1],
              [1, 0, 1, 1],
              [0, 1, 1, 1]]) # prior

# (B) represents which connections are modulated by experimental conditions. Each of the 
# square matrices refers to task, pictures, and words respectively. The task doesn't modulate
# any connections here, but pictures and word modulate the self connection between all regions. 
B = np.array([[[0, 0, 0, 0],
               [0, 0, 0, 0],
               [0, 0, 0, 0],
               [0, 0, 0, 0]],
              
             [[1, 0, 0, 0],
              [0, 1, 0, 0],
              [0, 0, 1, 0],
              [0, 0, 0, 1]],
              
             [[1, 0, 0, 0],
              [0, 1, 0, 0],
              [0, 0, 1, 0],
              [0, 0, 0, 1]]]) # prior

# (C) indicates which regions are driven by each experimental condition. Columns are the 
# conditions and rows the brain regions. So, all four regions are driven by task but pictures
# and words are just modulatory inputs. 
C = np.array([[1, 0, 0],
              [1, 0, 0], 
              [1, 0, 0],
              [1, 0, 0]]) # prior

Now we specify the DCM for participant 1. 

First, we extract the "u" factors for each causal and mediating variable, as predetermined in the experiment MATLAB code. We will need these to estimate the state vector. Each entry in these "u" vectors corresponds to one timepoint.

In [10]:
u_task_path = 'u_task.csv'
u_task = np.genfromtxt(u_task_path, delimiter=',')

u_pics_path = 'u_pictures.csv'
u_pictures = np.genfromtxt(u_pics_path, delimiter=',')

u_words_path = 'u_words.csv'
u_words = np.genfromtxt(u_words_path, delimiter=',')

We will now compute the neural state vector for each of the 3200 timepoints, 3.6 seconds apart. For our neural model, we will use the simple implementation in (2) in part 1 of the tutorial papers. First, we will need to compute the connectivity matrix $J$, then we can use the given differential equation for $z$ to compute the neural state vector $z$. The equations are below for reference. 

ADD EQUATIONS

These will be computed together for each time point using the priors for each of the vectors. 

In [13]:
# state vec is a 3200 x 4 array
state_vec = np.empty((3200, 4))

for index in range(3200):
    t = index * 3.6

    # Define ODE
    def neural_state_ode(t, y, index): 
        J = A + B[1, :, :] * u_pictures[index] +  B[2, :, :] * u_words[index]
        dydt = np.matmul(J, np.array(y).T) + np.matmul(C, np.array([[u_task[index]], [u_pictures[index]], [u_words[index]]])).T
        return dydt[0, :]
        
    # Initial condition
    y0 = [0, 0, 0, 0]  
    
    # Solve ODE
    z = solve_ivp(neural_state_ode, (t-1, t), y0, args=(index,))
    
    # Add to state vec
    state_vec[index, :] = z.y[:, -1]

print("state", state_vec)

state [[ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 ...
 [-1.79720564 -1.79720564 -1.79720564 -1.79720564]
 [-1.79720564 -1.79720564 -1.79720564 -1.79720564]
 [-1.79720564 -1.79720564 -1.79720564 -1.79720564]]


Now we use the hemodynamic equations to convert this estimated neural state vector into a BOLD fMRI signal. We start with the vasodilatory signal and flow equations, which are co-dependent. 

ADD EQUATIONS!!

In [16]:
kappa = 0 # Rate of vasodilatory signal decay (prior)
gamma = 6 # Rate of decay to feedback to vasodilatory signal (prior)

vasodilatory_vec = np.empty((3200, 4))
flow_vec = np.empty((3200, 4))

def vasodilatory_ode(t, s, x, f, kappa, gamma):
    dsdt = x - kappa * s - gamma * (f - np.ones(4))
    return dsdt

def flow_ode(t, f, s):
    dfdt = s
    return dfdt

f = [0, 0, 0, 0]
s = [0, 0, 0, 0]

for index in range(3200):
    t = index * 3.6
    
    # Iteratively determine s and f, starting with s
    s0 = [0, 0, 0, 0]
    f0 = [0, 0, 0, 0]

    # Solve ODE
    if index == 0:
        s = solve_ivp(vasodilatory_ode, (t-1, t), s0, args=(state_vec[index, :], f0, kappa, gamma))
        f = solve_ivp(flow_ode, (t-1, t), f0, args=(s0,))
    else:
        s = solve_ivp(vasodilatory_ode, (t-1, t), s0, args=(state_vec[index, :], f.y[:, -1], kappa, gamma))
        f = solve_ivp(flow_ode, (t-1, t), f0, args=(s.y[:, -1],))
    
    # Add to vecs...
    vasodilatory_vec[index, :] = s.y[:, -1]
    flow_vec[index, :] = f.y[:, -1]

print(vasodilatory_vec)
print(flow_vec)

/opt/anaconda3/lib/python3.12/site-packages/scipy/integrate/_ivp/common.py:108: RuntimeWarning: overflow encountered in divide
  d1 = norm(f0 / scale)
/var/folders/1c/my2535l97nv5xg5kfyw1f6680000gn/T/ipykernel_1456/2553106727.py:8: RuntimeWarning: invalid value encountered in multiply
  dsdt = x - kappa * s - gamma * (f - np.ones(4))


[[ 6.00000000e+00  6.00000000e+00  6.00000000e+00  6.00000000e+00]
 [ 6.00000000e+00  6.00000000e+00  6.00000000e+00  6.00000000e+00]
 [-3.00000000e+01 -3.00000000e+01 -3.00000000e+01 -3.00000000e+01]
 ...
 [ 2.21221006e+22  2.21221006e+22  2.21221006e+22  2.21221006e+22]
 [-1.32732603e+23 -1.32732603e+23 -1.32732603e+23 -1.32732603e+23]
 [ 7.96395620e+23  7.96395620e+23  7.96395620e+23  7.96395620e+23]]
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 6.00000000e+00  6.00000000e+00  6.00000000e+00  6.00000000e+00]
 [-3.00000000e+01 -3.00000000e+01 -3.00000000e+01 -3.00000000e+01]
 ...
 [ 2.21221006e+22  2.21221006e+22  2.21221006e+22  2.21221006e+22]
 [-1.32732603e+23 -1.32732603e+23 -1.32732603e+23 -1.32732603e+23]
 [ 7.96395620e+23  7.96395620e+23  7.96395620e+23  7.96395620e+23]]


Then we use the flow vector and neural state vectors to compute the change in volume and deoxyhemoglobin associated with the balloon hemodynamic model. First, the volume change ODE.

In [19]:
tau = 2      # hemodynamic transit time (seconds, static)
alpha = 0.32 # Grubb's exponent (blood vessel stiffness, static)

volume_vec = np.empty((3200, 4))

# Runtime issue here...
def volume_ode(t, v, f, alpha, tau):
    dvdt = (1/tau) * (f - np.power(v,1/alpha))
    return dvdt

v0 = [0, 0, 0, 0]
for index in range(3200):
    print(index, "out of 3200")
    t = index * 3.6

    # Solve ODE
    v = solve_ivp(volume_ode, (t-1, t), v0, args=(flow_vec[index,:], alpha, tau))

    volume_vec[index, :] = v.y[:, -1]

print(volume_vec)

0 out of 3200
1 out of 3200
2 out of 3200
3 out of 3200
4 out of 3200
5 out of 3200
6 out of 3200
7 out of 3200
8 out of 3200
9 out of 3200


/var/folders/1c/my2535l97nv5xg5kfyw1f6680000gn/T/ipykernel_1456/3423001994.py:8: RuntimeWarning: invalid value encountered in power
  dvdt = (1/tau) * (f - np.power(v,1/alpha))


10 out of 3200
11 out of 3200
12 out of 3200
13 out of 3200


KeyboardInterrupt: 

Then the deoxyhemoglobin change ODE. 

In [21]:
E0 = 0.4 # Resting oxygen extraction fraction (static)

# For a single region at a single timepoint... 
def dHb_ode(t, q, v, f, E0, alpha, tau):
    if v == 0 or tau == 0 or E0 == 0 or f == 0:
        raise ValueError("Denominator in the ODE is zero")
    dqdt = (1/tau) * ((f/E0) * (1 - (1 - E0)^(1/f)) - v^(1/alpha) * (q/v)) # Raising constant to a vector
    return dqdt

dHb_vec = np.empty((3200, 4))

q0 = [0]
for region in range(4):
    for index in range(3200):
        t = index * 3.6

        # Solve ODE
        q = solve_ivp(dHb_ode, (t-1, t), q0, args=(volume_vec[index, region], flow_vec[index, region], E0, alpha, tau))

        dHb_vec[index, region] = q.y[:, -1]

print(dHb_vec)

ValueError: Denominator in the ODE is zero

Then the BOLD signal change equation to get the change in signal above baseline produced by each region. This provides the basis of the BOLD timeseries we will compare with the experimental data. 

In [ ]:
TE = 0.04     # fMRI echo time (static)
epsilon_h = 0 # Fraction of intravascular to extravascular signal (prior)
r0 = 25       # Oxygen extraction rate constant (static)
nu0 = 40.3    # Blood volume normalized to rest (static)
V0 = 0        # Resting venous blood volume fraction (static)

k1 = 4.3 * nu0 * E0 * TE
k2 = epsilon_h * r0 * E0 * TE
k3 = 1 - epsilon_h

dS_S0 = V0 * (k1*(1 - dHb_vec) + k2*(1 - np.divide(dHb_vec, volume_vec)) + k3*(1 - volume_vec))

Now, we form the BOLD signal timeseries by adding in several more parameters to the equation: 

$y = g(z, \theta^h) + X_0 \beta_0 + \epsilon$

where $g(z, \theta^h)$ is what we just computed, $X_0$ is the design matrix for null effects and $\beta_0$ associated parameters, and $\epsilon$ is normally distributed observation noise.

For our purposes, we will assume that $X_0$ and $\beta_0$ are null, so we won't need to consider their effects. We will also not consider the effects of $\epsilon$, since our approximated data is normally distributed. 